In [41]:
import numpy as np
import pandas as pd
import tkinter.filedialog as fd
from tkinter import messagebox
import webbrowser
import os
import tkinter as tk
from tkinter import ttk
import itertools

In [3]:
import numpy as np
import pandas as pd
import tkinter.filedialog as fd
from tkinter import messagebox
import webbrowser
import os
import tkinter as tk
from tkinter import ttk

class MainApplication(tk.Tk):
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        # Add title to window
        self.wm_title("School Asset Portfolio Reducible Space Calculator")
        
        #self.geometry("500x400") #<-- this hardcodes the shape of the window but don't need that at this stage
        
        # Create frame and assign it to a container:
        container = tk.Frame(self)
        container.pack(side="top", fill="both", expand=True)
        
        # Configure the location of the container using grid
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)
        
        self.frames = {}
        
        for page in (PageOne, PageTwo):
            frame = page(parent = container, controller = self)
            
            # the windows class acts as root window for the frame
            self.frames[page] = frame
            frame.grid(row = 0, column = 0) # = "top", fill = "both", expand = True)
        self.show_frame(PageOne)
        
    #def show_frame(self, container):
        #frame = self.frames[container]
        #frame.tkraise()
        
    def show_frame(self, page_class):
        for frame in self.frames.values():
            frame.grid_remove()  # Hide all frames
        frame = self.frames[page_class]
        frame.grid(row=0, column=0)  # Show the desired frame
        frame.tkraise()  # Raise the frame to the front


class PageOne(tk.Frame):
    def __init__(self, parent, controller): # *args, **kwargs
        
        tk.Frame.__init__(self, parent)
        
        # This is the button to show page 2
        self.page_2_button = tk.Button(self,
                                       text="Page 2 >",
                                       command = lambda: controller.show_frame(PageTwo))
        self.page_2_button.grid(row=6, column=0, padx=10, pady=10)
        
        self.parent = parent
        self.controller = controller
        
        self.fas_upload_btn = None
        self.page_2_button = None
        
        self.fas_path = ""
        self.campus_properties_path = ""
        self.building_properties_path = ""
        self.aims_report_path = ""
        self.incl_excl_file_path = ""
        
        self.FAS_1_Categories = None
        self.FAS_2_Categories = None
        self.Primary_FAS = None
        self.Secondary_FAS = None
        
        self.campus_properties_df = None
        self.building_properties_df = None
        self.aims_report_df = None
        self.inclusion_exclusion_df = None

        self.create_widgets()
        self.classification_index = None
      
    
    def show_dataframe(self, df):
        # Create a new window
        window = tk.Toplevel()
        window.title("Dataframe Table")
        window.geometry("800x600") # Set the size of the window

        # Create a Treeview widget to display the dataframe
        table = ttk.Treeview(window, columns=list(df.columns), show="headings")
        for col in df.columns:
            table.heading(col, text=col)
        for i, row in df.iterrows():
            table.insert("", "end", values=list(row))

        # Add the Treeview widget to the window
        table.pack(fill="both", expand=True)

        # Start the tkinter mainloop
        window.mainloop()
    
    def create_widgets(self):
        
        # These are the buttons that upload the files
        self.fas_upload_btn = tk.Button(self, text="Upload FAS folder", command=self.upload_fas)
        self.fas_upload_btn.grid(row=0, column=0, padx=10, pady=10)
        self.fas_desc_btn = tk.Button(self, text="Description", command=self.show_fas_description)
        self.fas_desc_btn.grid(row=0, column=1, padx=10, pady=10)
        
        self.campus_upload_btn = tk.Button(self, text="Upload Campus Properties CSV", command=self.upload_campus_properties)
        self.campus_upload_btn.grid(row=1, column=0, padx=10, pady=10)
        self.campus_desc_btn = tk.Button(self, text="Description", command=self.show_campus_properties_description)
        self.campus_desc_btn.grid(row=1, column=1, padx=10, pady=10)
        
        self.building_upload_btn = tk.Button(self, text="Upload Building Properties CSV", command=self.upload_building_properties)
        self.building_upload_btn.grid(row=2, column=0, padx=10, pady=10)
        self.building_desc_btn = tk.Button(self, text="Description", command=self.show_building_properties_description)
        self.building_desc_btn.grid(row=2, column=1, padx=10, pady=10)
        
        self.aims_upload_btn = tk.Button(self, text="Upload AIMS Building-Space Report CSV", command=self.upload_aims_report)
        self.aims_upload_btn.grid(row=3, column=0, padx=10, pady=10)
        self.aims_desc_btn = tk.Button(self, text="Description", command=self.show_aims_report_description)
        self.aims_desc_btn.grid(row=3, column=1, padx=10, pady=10)
        self.aims_link_button = tk.Button(self, text = "Link to AIMS extract", command=self.link_to_AIMS_extract)
        self.aims_link_button.grid(row=3, column = 2, padx=10, pady=10)
        
        self.inclusion_exclusion_upload_btn = tk.Button(self, text="Upload Inclusion-Exclusion Register CSV", command=self.upload_inclusion_exclusion)
        self.inclusion_exclusion_upload_btn.grid(row=4, column=0, padx=10, pady=10)
        self.inclusion_exclusion_desc_btn = tk.Button(self, text="Description", command=self.show_inclusion_exclusion_description)
        self.inclusion_exclusion_desc_btn.grid(row=4, column=1, padx=10, pady=10)
        
    
    """
    
    This section converts the 4 FAS files into the required formats:
    
    """
    
    # 1: Button calls this function which retrieves the 4 FAS files and return them as a dictionary
    def upload_fas(self):
        fas_folder_path = fd.askdirectory()       
        fas_folder_path = fas_folder_path.replace('\\', '/')

        file_names = ['FAS 1 Categories.csv', 'FAS 2 Categories.csv', 'Primary FAS.csv', 'Secondary FAS.csv']
        FAS_dataframes = { }

        for file in os.listdir(fas_folder_path):
            for idx, target_file in enumerate(file_names):
                if file == target_file:
                    file_path = os.path.join(fas_folder_path, file)
                    df = pd.read_csv(file_path)
                    FAS_dataframes[target_file] = df                 
                    break
                      
        """Below is the method for processing the FAS1-2 Category dataframes"""
        
        FAS_1_categories = FAS_dataframes['FAS 1 Categories.csv']
        FAS_2_categories = FAS_dataframes['FAS 2 Categories.csv']
        FAS_1_categories = FAS_1_categories.set_index("Facility Area Schedule")
        FAS_2_categories = FAS_2_categories.set_index("Facility Area Schedule")
        
        self.FAS_1_categories = FAS_1_categories
        self.FAS_2_categories = FAS_2_categories
        
        
        """Below is the method for processing the Pri_FAS & Sec_FAS dataframes"""
        
        Pri_FAS = FAS_dataframes['Primary FAS.csv']
        Sec_FAS = FAS_dataframes['Secondary FAS.csv']

        # 2: rename columns to match later requirements
        Sec_FAS.rename(columns = {'Upper Enrolments': 'FAS Layer 2', 'FAS Level' : 'FAS Layer'}, inplace = True)
        Pri_FAS.rename(columns = {'Upper Enrolments': 'FAS Layer 2'}, inplace = True)

        # 3: Set the category names as the indices
        Sec_FAS_index = Sec_FAS['FAS Layer 2']
        Sec_FAS = Sec_FAS.set_index(Sec_FAS_index)
        Pri_FAS_index = Pri_FAS['FAS Layer 2']
        Pri_FAS = Pri_FAS.set_index(Pri_FAS_index)

        # 4: remove FAS categories column
        Sec_FAS = Sec_FAS.drop(['FAS Layer 2'], axis = 1)
        Pri_FAS = Pri_FAS.drop(['FAS Layer 2'], axis = 1)

        # 5: insert an enrolment band for '0' students
        Pri_FAS.insert(1, '0', 0)
        Sec_FAS.insert(1, '0', 0)

        # 6: insert a column classifying which enrolment type the row belongs to

        Pri_FAS.insert(1, 'Student Type', 'Primary')

        Sec_FAS.insert(1, 'Student Type', 'Secondary')

        # 7: Splits the indices into a multi-index depending on FAS Layer

        def split_index(FAS_df):
            FAS_1 = None
            new_index = []

            for n, index in enumerate(FAS_df.index):
                layer = FAS_df.iat[n, 0]

                if layer == 1:
                    FAS_1 = index
                elif layer == 2:
                    new_index.append((FAS_1, index))

            FAS_df = FAS_df[FAS_df['FAS Layer'] > 1]
            FAS_df.set_index(pd.MultiIndex.from_tuples(new_index, names=['FAS 1', 'FAS 2']), inplace=True)
            FAS_df = FAS_df.drop(columns=["FAS Layer"])

            return FAS_df

        Pri_FAS = split_index(Pri_FAS)

        Sec_FAS = split_index(Sec_FAS)

        # 8: Assigns the Core/Support label to each row (FAS Category)

        def core_support(FAS_df, FAS_2_categories):
            core_support_column = []
            FAS_1_values = FAS_df.index.get_level_values(0).unique()
            FAS_2_values = FAS_df.index.get_level_values(1)
            for FAS_2 in FAS_2_values:
                if FAS_2_categories.at[FAS_2, "Space classification"] == 'Core':
                    core_support_column.append('Core')
                elif FAS_2_categories.at[FAS_2, "Space classification"] == 'Support':
                    core_support_column.append('Support')   
                else:
                    core_support_column.append('N/A')
            FAS_df.insert(0, "Space classification", core_support_column)
            return FAS_df
        
        self.Pri_FAS = core_support(Pri_FAS, FAS_2_categories)
        self.Sec_FAS = core_support(Sec_FAS, FAS_2_categories)
        
        FAS_dataframes_dict = { 'Pri_FAS' : self.Pri_FAS, 'Sec_FAS' : self.Sec_FAS, 'FAS_1_categories' : self.FAS_1_categories, 'FAS_2_categories' : self.FAS_2_categories }
        
        return FAS_dataframes_dict
        
    """
    ============================================================================
    """
    
    # This function is used in the import functions to convert the dataframes 
    def convert_columns(self, dataframe, data_types):
        for column, dtype in data_types.items():
            if column in dataframe.columns:
                try:
                    if dtype == int or dtype == float:
                        dataframe[column] = pd.to_numeric(dataframe[column], errors='coerce')
                        dataframe[column].fillna(np.nan, inplace=True)
                    elif dtype == str:
                        dataframe[column] = dataframe[column].astype(str).replace('nan', '', regex=True)
                    else:
                        raise ValueError(f"Unsupported datatype '{dtype}' for column '{column}'.")
                except Exception as e:
                    error_msg = f"Error while converting column '{column}' to datatype '{dtype}':\n\n{str(e)}"
                    tk.messagebox.showerror(title="Error", message=error_msg)
                    raise
        return dataframe
    
    def upload_campus_properties(self):
        filetypes = (("CSV files", "*.csv"), ("All files", "*.*"))
        campus_properties_data_types = {'CampusId' : str,
                                        'School Type' : str,
                                        'primary_enrolments' : int,
                                        'secondary_enrolments' : int }

        self.campus_properties_path = fd.askopenfilename(title = "Select Campus Properties CSV file", filetypes=filetypes)
        self.check_csv_columns(self.campus_properties_path, ["CampusId", "primary_enrolments", "secondary_enrolments"])
        if self.campus_properties_path != "":
            try:
                self.campus_properties_df = pd.read_csv(self.campus_properties_path)
            except:
                tk.messagebox.showerror(title="Error", message=f"Failed to read file: {self.campus_properties_path}")
        self.convert_columns(self.campus_properties_df, campus_properties_data_types)
        self.show_dataframe(self.campus_properties_df)
        
        
    def upload_building_properties(self):
        building_properties_data_types = { 'CampusId' : str, 'BuildingId' : str, 'Condition' : float , 'Functionality' : float }
        filetypes = (("CSV files", "*.csv"), ("All files", "*.*"))
        self.building_properties_path = fd.askopenfilename(title="Select Building Properties CSV file", filetypes=filetypes)
        self.check_csv_columns(self.building_properties_path, ["BuildingId", "CampusId", "Condition", "Functionality"])
        if self.building_properties_path != "":
            try:
                self.building_properties_df = pd.read_csv(self.building_properties_path, dtype = building_properties_data_types)
                print(self.building_properties_df.columns)
                      
            except:
                tk.messagebox.showerror(title="Error", message=f"Failed to read file: {self.building_properties_path}")
        self.convert_columns(self.building_properties_df, building_properties_data_types)
        self.show_dataframe(self.building_properties_df)
 
    """
    This this the code for cleaning the asset data and converting it into the FAS 2 FloorArea for each building
    """
    
    def upload_aims_report(self):
        aims_report_data_types = { 'FloorArea' : float, 'CampusId' : str, 'BuildingId' : str, 'SpaceEntClass18' : str }
        filetypes = (("CSV files", "*.csv"), ("All files", "*.*"))
        self.aims_report_path = fd.askopenfilename(title="Select AIMS Building-Space Report CSV file", filetypes=filetypes)
        self.check_csv_columns(self.aims_report_path, ["SpaceEntClass18", "CampusId", "FloorArea", "BuildingId"])
        if self.aims_report_path != "":
            try:
                self.aims_report_df = pd.read_csv(self.aims_report_path, dtype = aims_report_data_types)
                      
            except:
                tk.messagebox.showerror(title="Error", message=f"Failed to read file: {self.aims_report_path}")
                
                
        actual_space_extract = self.aims_report_df.loc[:,['CampusId', 'SchoolNo', 'CampusName', 'BuildingId', 'BuildingName', 'SpaceId','SpaceEntClass18', 'FloorArea', 'Classification']]

        
        # This produces a dataframe with all the unique SpaceEntClass18 values broken into the constituent values e.g. so that they can be matched to FAS 1 & 2 categories
        space_classifications = self.aims_report_df['SpaceEntClass18'].unique()
        self.classification_index = pd.DataFrame(space_classifications, columns=['space_classifications']).set_index('space_classifications')
        self.classification_index[['year', 'standard', 'FAS 1', 'FAS 2', 'FAS 3']] = self.classification_index.index.to_series().str.split('\\\\', expand=True)     
           
        # This converts the columns to the right data type
        self.convert_columns(self.aims_report_df, aims_report_data_types)
        
        # filtering out any rows with null / not numerical values for floor area
        actual_space_extract = actual_space_extract.loc[(actual_space_extract["FloorArea"] > -1)]

        # this assigns two new columns to the actual_space_extract file
        actual_space_extract = actual_space_extract.assign(**{"FAS 1": "", "FAS 2": ""})
        
        # Uses 'map' to retrieve the FAS 1 & 2 categories from the classification index according to the SpaceEntClass18 value in the AIMS extract
        actual_space_extract['FAS 1'] = actual_space_extract['SpaceEntClass18'].map(self.classification_index['FAS 1'])
        actual_space_extract['FAS 2'] = actual_space_extract['SpaceEntClass18'].map(self.classification_index['FAS 2'])
        
        # Uses `str.strip()` to remove leading and trailing spaces from the columns
        actual_space_extract['FAS 1'] = actual_space_extract['FAS 1'].str.strip()
        actual_space_extract['FAS 2'] = actual_space_extract['FAS 2'].str.strip()
        
        self.aims_report_df = actual_space_extract
        
        core_space_categories = self.Pri_FAS[self.Pri_FAS['Space classification'] == 'Core'].index.get_level_values(1).tolist() + self.Sec_FAS[self.Sec_FAS['Space classification'] == 'Core'].index.get_level_values(1).tolist()
        
        self.FAS_2_space = self.aims_report_df.pivot_table(index = 'BuildingId', columns = 'FAS 2',
                                                          values='FloorArea',  aggfunc = 'sum',  fill_value = 0)
        
        self.FAS_1_space = self.aims_report_df.pivot_table(index = 'BuildingId', columns = 'FAS 1',
                                                          values='FloorArea',  aggfunc = 'sum',  fill_value = 0)
        
        self.FAS_2_core_space = self.FAS_2_space.filter(items = core_space_categories)
        
        # self.show_dataframe(pd.DataFrame(self.FAS_2_core_space.columns)) This shows the columns in the FAS_2_core dataframe
        
        
    def upload_inclusion_exclusion(self):
        filetypes = (("CSV files", "*.csv"), ("All files", "*.*"))
        inclusion_exclusion_data_types = { 'filter_true_false' : bool,
                                          'exclusion_type' : str,
                                          'num_range_start' : float,
                                          'num_range_stop' : float }
        self.inclusion_exclusion_path = fd.askopenfilename(title="Select Inclusion-Exclusion Register CSV file", filetypes=filetypes)
        self.check_csv_columns(self.inclusion_exclusion_path,
                               ["table_name", "column_name",
                                "exclude_true_false", "exclusion_type",
                                "num_range_start", "num_range_stop", "string_value_criteria"])
        if self.inclusion_exclusion_path != "":
            try:
                self.inclusion_exclusion_df = pd.read_csv(self.inclusion_exclusion_path)
                print(self.inclusion_exclusion_df.columns)
            except:
                tk.messagebox.showerror(title="Error", message=f"Failed to read file: {self.inclusion_exclusion_path}")
        self.convert_columns(self.inclusion_exclusion_df, inclusion_exclusion_data_types)
        self.show_dataframe(self.inclusion_exclusion_df)
        
    """
    =====================================================================================
    """
        
    def check_csv_columns(self, file_path, required_columns):
        if file_path != "":
            try:
                df = pd.read_csv(file_path)
                columns = df.columns.tolist()
                for col in required_columns:
                    if col not in columns:
                        tk.messagebox.showwarning(title="Missing Column", message=f"File {file_path} is missing the required column: {col}")
                        break
            except:
                tk.messagebox.showerror(title="Error", message=f"Failed to read file: {file_path}")
                
    
    def show_fas_description(self):
        tk.messagebox.showinfo(title="FAS CSV Description",
                               message="The FAS is split into 4 different csv files, which need to be uploaded simultaneously by selecting the folder that they are contained in. This should be a dedicated folder that contains no other files. In the case that changes are made to the Facility Area Schedule, these changes should be updated in ALL the CSV files, otherwise errors might occur.\n\nThe 4 csv files MUST be named 'FAS 1 Categories', 'FAS 2 Categories', 'Primary FAS', 'Secondary FAS'. ")
    
    def show_campus_properties_description(self):
        tk.messagebox.showinfo(title="Campus Properties CSV Description",
                               message="Columns contain enrolment information for each school campus, as well as properties that are used as inclusion/exclusion criteria.\n\nMust include a column called 'CampusId', 'primary_enrolments', 'secondary_enrolments'.")
    
    def show_building_properties_description(self):
        tk.messagebox.showinfo(title="Building Properties CSV Description",
                               message="Columns contain any properties for buildings that are used in inclusion/exclusion criteria.\n\nMust include columns called “BuildingId”, 'CampusId', 'Condition', 'Functionality'.")
    
    def show_aims_report_description(self):
        tk.messagebox.showinfo(title="AIMS Building-Space Report CSV Description",
                               message="Provides FAS1/2 floor area per building.\nMust include a columns called: “SpaceEntClass18”, “CampusID”, “FloorArea” and “BuildingId”.\n\nCan be downloaded from this link: https://partner.eduweb.vic.gov.au/collaboration/AIMS/S1R1%20CORP/Forms/DET%20Document%20Set/docsethomepage.aspx?RootFolder=%2Fcollaboration%2FAIMS%2FS1R1%20CORP%2FS1%20R1%20AIMS%20REPORT%20EXAMPLES%2FRPT001%20-%20Extract&FolderCTID=0x0120007D803ED7D3E25A4587B6EED2CA7F9FB5&View=%7B8E2F392E-4991-44AF-9300-0D91D42638C7%7D’")

    def show_inclusion_exclusion_description(self):
        tk.messagebox.showinfo(title = "Inclusion-Exclusion Register CSV Description",
                               message = "Provides a register of the exclusion criteria. The register must consist of the following columns:\n1. “table_name” (the name of the table that the column is in),\n2. “column_name” (the name of the column for the criteria),\n3. “exclude_true_false” (determines whether the criteria is active: True = active, False = NOT active),\n“4. exclusion_type” (determines the type of exclusion criteria; valid entries are “num_range”  and “string_value”), IF exclusion type is 'num_range', the the following two columns are used: 'num_range_start' AND 'num_range_stop' which indicates the beginning and end of the exclusion range. IF the exclusion type is 'string_value', then  criteria can be written as one or more string values, with multiple values separated by commas (e.g  Building123,Building435).")
    
    def link_to_AIMS_extract(self):
        webbrowser.open("https://partner.eduweb.vic.gov.au/collaboration/AIMS/S1R1%20CORP/Forms/DET%20Document%20Set/docsethomepage.aspx?RootFolder=%2Fcollaboration%2FAIMS%2FS1R1%20CORP%2FS1%20R1%20AIMS%20REPORT%20EXAMPLES%2FRPT001%20-%20Extract&FolderCTID=0x0120007D803ED7D3E25A4587B6EED2CA7F9FB5&View=%7B8E2F392E-4991-44AF-9300-0D91D42638C7%7D")
        

class PageTwo(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Page Two")

        page_1_button = ttk.Button(self,
                                   text="Back to Page One",
                                   command = lambda: controller.show_frame(PageOne))
        page_1_button.pack(side="bottom", fill=tk.X)
        

if __name__ == "__main__":
    app = MainApplication()
    app.mainloop()

C:\Users\10349839\AppData\Local\Temp\ipykernel_5440\4072559011.py:383: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\10349839\AppData\Local\Temp\ipykernel_5440\4072559011.py:291: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  self.building_properties_df = pd.read_csv(self.building_properties_path, dtype = building_properties_data_types)


Index(['CampusId', 'SchoolNo', 'CampusNo', 'CampusName', 'SiteName',
       'BuildingFunc', 'BuildingUse', 'BuildingId', 'AltName', 'BuildingName',
       'Relocatable', 'RelocMods', 'FloorNo', 'SpaceId', 'RoomNo',
       'AltSpaceName', 'SpaceCleanClass12', 'SpaceEntClass12',
       'SpaceCleanClass18', 'SpaceEntClass18', 'FloorArea', 'CampusType',
       'PrimaryKey', 'BuildCreateDate', 'Classification', 'BuildingLowStorey',
       'BuildingHighStorey', 'BuildingComment', 'SpaceComment', 'SchoolType',
       'BuildingLoc', 'AtriumBlockId', 'AtriumLocationId',
       'AbbrSpCleanClass12', 'AbbrSpEntClass12', 'AbbrSpCleanClass18',
       'AbbrSpEntClass18', 'SpaceCategory', 'Description'],
      dtype='object')


In [2]:
aims_report_data_types = { 'FloorArea' : float, 'CampusId' : str, 'BuildingId' : str, 'SpaceEntClass18' : str }

asset_extract = pd.read_csv(r'ASSETS_20230310.csv', dtype = aims_report_data_types)

# This produces a dataframe with all the unique SpaceEntClass18 values broken into the constituent values e.g. so that they can be matched to FAS 1 & 2 categories
space_classifications = asset_extract['SpaceEntClass18'].unique()
classification_index = pd.DataFrame(space_classifications, columns=['space_classifications']).set_index('space_classifications')
classification_index[['year', 'standard', 'FAS 1', 'FAS 2', 'FAS 3']] = classification_index.index.to_series().str.split('\\\\', expand=True)     

# filtering the columns that are needed
actual_space_extract = asset_extract.loc[:,['CampusId', 'SchoolNo', 'CampusName', 'CampusType', 'BuildingFunc', 'BuildingUse', 'BuildingId', 'BuildingName',
       'Relocatable', 'SpaceId','SpaceEntClass18', 'FloorArea', 'Classification']]

# filtering out any rows with null / not numerical values for floor area
actual_space_extract = actual_space_extract.loc[(actual_space_extract["FloorArea"] >= 0)]

# this assigns two new columns to the actual_space_extract file
actual_space_extract = actual_space_extract.assign(**{"FAS 1": "", "FAS 2": ""})

actual_space_extract['FAS 1'] = actual_space_extract['SpaceEntClass18'].map(classification_index['FAS 1'])
actual_space_extract['FAS 2'] = actual_space_extract['SpaceEntClass18'].map(classification_index['FAS 2'])

# Use `str.strip()` to remove leading and trailing spaces from the columns
actual_space_extract['FAS 1'] = actual_space_extract['FAS 1'].str.strip()
actual_space_extract['FAS 2'] = actual_space_extract['FAS 2'].str.strip()

actual_space_extract

FAS_2_space = actual_space_extract.pivot_table(index = 'BuildingId', columns = 'FAS 2', values='FloorArea', aggfunc = 'sum', fill_value = 0)

FAS_1_space = actual_space_extract.pivot_table(index = 'BuildingId', columns = 'FAS 1', values='FloorArea', aggfunc = 'sum', fill_value = 0)



C:\Users\10349839\AppData\Local\Temp\ipykernel_10720\1004297712.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  asset_extract = pd.read_csv(r'ASSETS_20230310.csv', dtype = aims_report_data_types)


In [109]:
actual_space_extract.loc[(actual_space_extract["CampusId"] == '1883001') & (actual_space_extract["FAS 1"] == 'Technology')]

,CampusId,SchoolNo,CampusName,CampusType,BuildingFunc,BuildingUse,BuildingId,BuildingName,Relocatable,SpaceId,SpaceEntClass18,FloorArea,Classification,FAS 1,FAS 2
134910,1883001,8830,Charlton College,GSP-PRISEC,CURRICULUM,BUILDING,105587,Charlton:Davies - TTC,N,329744,2018 \ Secondary-STD \ Technology \ Materials ...,59.0,BUILDING,Technology,Materials Technology
134915,1883001,8830,Charlton College,GSP-PRISEC,CURRICULUM,BUILDING,105587,Charlton:Davies - TTC,N,329749,2018 \ Secondary-STD \ Technology \ Materials ...,5.0,BUILDING,Technology,Materials Technology
134916,1883001,8830,Charlton College,GSP-PRISEC,CURRICULUM,BUILDING,105587,Charlton:Davies - TTC,N,329750,2018 \ Secondary-STD \ Technology \ Materials ...,5.0,BUILDING,Technology,Materials Technology
134917,1883001,8830,Charlton College,GSP-PRISEC,CURRICULUM,BUILDING,105587,Charlton:Davies - TTC,N,329751,2018 \ Secondary-STD \ Technology \ Materials ...,5.0,BUILDING,Technology,Materials Technology
138418,1883001,8830,Charlton College,GSP-PRISEC,CURRICULUM,BUILDING,105304,Charlton:Davies - Main Classroom Wing,N,333963,2018 \ Secondary-STD \ Technology \ Integrated...,33.0,BUILDING,Technology,Integrated Materials Technology Learning
138921,1883001,8830,Charlton College,GSP-PRISEC,CURRICULUM,BUILDING,105304,Charlton:Davies - Main Classroom Wing,N,334576,2018 \ Secondary-STD \ Technology \ Integrated...,36.0,BUILDING,Technology,Integrated Materials Technology Learning
142031,1883001,8830,Charlton College,GSP-PRISEC,CURRICULUM,BUILDING,105499,Charlton:Davies - BER Slc,N,338467,2018 \ Secondary-STD \ Technology \ Integrated...,31.0,BUILDING,Technology,Integrated Materials Technology Learning
208032,1883001,8830,Charlton College,GSP-PRISEC,CURRICULUM,BUILDING,105587,Charlton:Davies - TTC,N,458305,2018 \ Secondary-STD \ Technology \ Materials ...,5.0,BUILDING,Technology,Materials Technology
234274,1883001,8830,Charlton College,GSP-PRISEC,CURRICULUM,BUILDING,105304,Charlton:Davies - Main Classroom Wing,N,543791,2018 \ Secondary-STD \ Technology \ Materials ...,2.6,BUILDING,Technology,Materials Technology


In [29]:
for area, building in zip(actual_space_extract['FloorArea'], actual_space_extract['SpaceId']):
    if not isinstance(area, float):
        print(area, type(area), building)

In [137]:
core_space_categories = ['Shared Learning', 'Practical Activities', 'Consult/Individual Learning Space',
                         'Support Spaces (Kitchen, Laundry, Equipment Store)', 'Relocatable Buildings - Learning Base',
                         'Practical Activities - Art/Science/Technology/Food Preparation', 'Practical Activities - Science/Technology/Food Preparation', 'Practical Activities - Art',
                         'Practical Activities - Science/Technology', 'Practical Activities - Food Preparation', 'Relocatable Buildings - Resource Intensive Learning', 'Multi-Purpose Space',
                         'Physical Education Gymnasium', 'Music', 'Canteen', 'Relocatable Buildings - Music', 'Relocatable Buildings - Canteen', 'Library', 'Senior Personnel Work Area',
                         'Support Spaces/Meeting Rooms', 'Staff Centre', 'Relocatable Buildings - Staff Centre', 'First Aid', 'Staff Work', 'Staff Amenities (Toilets, Shower, Change)',
                         'Student Amenities (Toilets, Shower, Change)', 'Shared Learning', 'Practical Activities', 'Consult/Individual Learning Space', 'Support Spaces (Kitchen, Laundry, Equipment Store)',
                         'Relocatable Buildings - Learning Base', 'Visual Arts', 'Textiles', 'Ceramics/Sculpture', 'Media Arts', 'Visual Communication', 'Integrated Arts Learning', 'Relocatable Buildings - The Arts',
                         'Materials Technology', 'Integrated Materials Technology Learning', 'Relocatable Buildings - Technology', 'Chemistry', 'Physics', 'Biology/General Science', 'Science Preparation/Storage',
                         'Integrated Science Learning', 'Relocatable Buildings - Science (Resource Intensive Science)', 'Relocatable Buildings - Science (Integrated Science Learning)',
                         'Physical Education Gymnasium', 'Music/Drama/Dance/Multi-Purpose Space', 'Canteen', 'Food Technology', 'Relocatable Buildings - Music and Drama', 'Relocatable Buildings - Food Tech',
                         'Relocatable Buildings - Canteen', 'Library', 'Senior Personnel Work Area', 'Support Spaces/Meeting Rooms', 'Staff Centre', 'First Aid', 'Relocatable Buildings - Staff Admin', 'Staff Work',
                         'Staff Amenities (Toilets, Shower, Change)', 'Student Amenities (Toilets, Shower, Change)', 'Relocatable Buildings - Toilets and Student Centre']

In [9]:
exclusion_df1 = pd.DataFrame({})
    
class Building(Campus):
    def __init__(self, building_id, building_properties, FAS_2_space, FAS_2_core_space, aims_extract):
        
        """
        Attributes are:
        
        1. self.FAS_2_space
        2. self.FAS_1_space
        3. self.FAS_1_core_space
        4. self.FAS_1_2_core_space
        
        """
        
        self.building_id = building_id
        self.exclusion_attributes = building_properties[building_properties['BuildingId'] == building_id]
        self.FAS_2_space = FAS_2_space.loc[[building_id]]        
        self.FAS_1_space = FAS_1_space.loc[[building_id]]
        
        
        # This returns an abbreviated version of the AIMS BRR report which only contains spaces which match the building Id
        self.aims_extract = aims_extract[aims_extract['BuildingId'] == building_id].filter(items = ['BuildingId', 'FAS 1', 'FAS 2', 'FloorArea'])
        
        
        # This finds the FAS 2 categories present in the current building
        FAS_2_building_categories = self.aims_extract['FAS 2'].unique()
        
        self.FAS_1_2_data = pd.DataFrame(index = FAS_2_building_categories, columns = ['FAS 1', 'Space Classification', 'Actual Space'])
        
        # This loop finds the FAS 1 category, floor area and Core/Support category for each FAS 2 category in the building
        for FAS_2 in FAS_2_building_categories:
            try:
                matching_FAS_1 = self.aims_extract.loc[self.aims_extract['FAS 2'] == FAS_2, 'FAS 1'].tolist()
                floor_area = self.FAS_2_space.at[building_id, FAS_2]
            # This handles buildings which randomly throw up errors (quite rare but seems to happen)
            except:
                continue
            try:
                matching_category = FAS_2_categories.at[FAS_2, 'Space classification'] # # # !!!!! FAS_2_Categories needs to changed to call on thr PageOne class
            except KeyError:
                matching_category = 'Support'
            
            self.FAS_1_2_data.at[FAS_2,'FAS 1'] = matching_FAS_1[0]
            self.FAS_1_2_data.at[FAS_2,'Actual Space'] = floor_area
            self.FAS_1_2_data.at[FAS_2,'Space Classification'] = matching_category
               
        self.FAS_1_2_core_space = self.FAS_1_2_data.loc[self.FAS_1_2_data['Space Classification'] == 'Core']
        
        for i in self.FAS_1_2_core_space['Actual Space']:
            if not isinstance(i, float):
                print(f"Value: {i}, type: {type(i)}, building: {building_id}")
        
        # This is to handle buildings with no core space (I.e. they have no columns 
        if len(self.FAS_1_2_core_space) > 0:
            self.FAS_1_core_space = self.FAS_1_2_core_space.pivot_table(index = 'FAS 1', values = 'Actual Space', aggfunc = 'sum')
        if len(self.FAS_1_2_core_space) < 1:
            self.FAS_1_core_space = self.FAS_1_2_core_space.set_index('FAS 1')
 
"""
Building attributes:
        1. self.FAS_2_space
        2. self.FAS_1_space
        3. self.FAS_1_core_space
        4. self.FAS_1_2_core_space
"""


building = Building('182350', actual_space_extract, FAS_2_space, FAS_1_space, actual_space_extract)

table = building.FAS_1_2_core_space

print(building.FAS_1_space.sum(axis=1))
print(building.FAS_2_space.sum(axis=1))

building.FAS_1_core_space

BuildingId
182350    1572.5
dtype: float64
BuildingId
182350    1572.5
dtype: float64


,Actual Space
FAS 1,
Administration,2.0
Learning Base,323.8
"Performing Arts, PE, Canteen, Food Technology",62.8
Staff Work,40.9
Student and Staff Amenities,16.2
Technology,13.7
The Arts - Fine Arts and Multimedia,522.1


In [95]:
table1 = table.iloc[2:4]

table2 = table.iloc[3:5]

table3 = table.iloc[3:6]

table_list = [table1, table2, table3]

merged = pd.concat(table_list, axis = 1, join = 'outer')
merged.fillna(0, inplace=True)
merged.fillna(0, inplace=True)
FAS1_df = merged.sum(axis = 1)
total_df = FAS1_df.sum()

print(table1,"\n", table2,"\n", table3)

                             Actual Space
FAS 1                                    
Resource Intensive Learning          26.0
School Community Hub                 55.1 
                       Actual Space
FAS 1                             
School Community Hub          55.1
Staff Work                    28.6 
                              Actual Space
FAS 1                                    
School Community Hub                 55.1
Staff Work                           28.6
Student and Staff Amenities          48.0


In [96]:
FAS1_df

FAS 1
Resource Intensive Learning     26.0
School Community Hub           165.3
Staff Work                      57.2
Student and Staff Amenities     48.0
dtype: float64

In [97]:
total_df

296.5

In [68]:
split_building_list = np.array_split(building_id_list, 500)

building_list_1 = split_building_list[0]

building_dictionary = {}

for b in building_list_1:
    try:
        building_dictionary[b] = Building(b, actual_space_extract, FAS_2_space, FAS_1_space, actual_space_extract)
    except:
        print(b)
        continue

C:\Users\10349839\AppData\Local\Temp\ipykernel_4128\4099973494.py:43: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  self.FAS_1_core_space = self.FAS_1_2_core_space.pivot_table(index = 'FAS 1', values = 'Actual Space', aggfunc = 'sum')
C:\Users\10349839\AppData\Local\Temp\ipykernel_4128\4099973494.py:43: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  self.FAS_1_core_space = self.FAS_1_2_core_space.pivot_table(index = 'FAS 1', values = 'Actual Space', aggfunc = 'sum')
C:\Users\10349839\AppData\Local\Temp\ipykernel_4128\4099973494.py:43: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Se

In [191]:
class Combination():
    def __init__(self, combination_tuple, FAS_1_excess_core_space, building_dictionary, FAS_1_core_entitlement, FAS_1_core_space, entitled_core_space):
               
        FAS_1_excess_core_space = FAS_1_excess_core_space.apply(lambda x: x if x >= 0 else 0) # <-- this applies a function which converts negative values to zero
                        
        combination_space = FAS_1_core_entitlement
        
        combination_space = combination_space.apply(lambda x: 0) # <-- creates a fresh pd.Series with all the FAS 1 categories and 0 for actual space
        
        # This loop finds the amount of space in each FAS 1 category
        for ID in combination_tuple:
            
            # retrieves the building object from the building dictionary created for the current campus object
            building = building_dictionary[ID]
            building_space = building.FAS_1_core_space
            
            print(f"ID: {ID}, building space: {building_space}")
                    
            # adds the building space to the pd.series representing the combination's space
            combination_space = combination_space.add(building_space, fill_value = 0)
            print(f"Combination space: {combination_space} \n=============================================\n")
                    
        
        # This is the "actual space" for the combination of buildings
        self.combination_FAS_1_core_space = combination_space
       
        self.FAS_1_remaining_excess_space = FAS_1_excess_core_space.sub(combination_space) # <--  subtracts the combination area from the excess core space to find the remaining excess space at each FAS 1 level
        
        self.FAS_1_remaining_core_space = FAS_1_core_space.sub(combination_space) #  <-- This subtracts the combination area from the core space to find the remaining space at each FAS 1 level
        
        self.remaining_total_space = self.FAS_1_remaining_core_space.sum() #  <-- This is a float value that shows the sum of core space
        
        # This finds the percentage of excess space remaining after the combination is removed
        actual = self.remaining_total_space
        entitled = entitled_core_space
        self.percentage_excess_core_space = ((actual - entitled) / actual ) * 100
        
        """ This section classifies the combination as reducible or not reducible"""
        
        if (self.FAS_1_remaining_core_space < 0).any():
            reducible_classification = False
        elif self.percentage_excess_core_space < 0:
            reducible_classification = False
        else:
            reducible_classification = True
            
        self.reducible_classification = reducible_classification
        
        self.reducible_space = self.combination_FAS_1_core_space.sum()


In [192]:
campus_properties_data_types = {'CampusId' : str,  'School Type' : str, 'primary_enrolments' : int, 'secondary_enrolments' : int }

campus_properties = pd.read_csv(r"C:\Users\10349839\Desktop\Reducible Space Calc\Test files\Campus Properties.csv", dtype = campus_properties_data_types)

inclusion_exclusion_data_types = { 'filter_true_false' : bool,
                                          'exclusion_type' : str,
                                          'num_range_start' : float,
                                          'num_range_stop' : float }

inclusion_exclusion_df = pd.read_csv(r"C:\Users\10349839\Desktop\Reducible Space Calc\Test files\Inclusion-Exclusion Register.csv", dtype = inclusion_exclusion_data_types)

building_properties_data_types = { 'CampusId' : str, 'BuildingId' : str, 'Condition' : float , 'Functionality' : float }

building_properties = pd.read_csv(r"C:\Users\10349839\Desktop\Reducible Space Calc\Test files\Building Properties - Example Schools.csv", dtype = building_properties_data_types)

# # # # #


class Campus():
    def __init__(self, campus_id, campus_properties, aims_extract, FAS_2_space, FAS_2_core_space, building_properties, inclusion_exclusion_df):
        
        self.enrolment_categories_tuple = self.enrolmentment_categories(campus_id, campus_properties, FAS_1_categories, FAS_2_categories, Sec_FAS, Pri_FAS) # # # !!! The FAS category data will need to have the PageOne class specified        
        
        buildings = building_properties[building_properties['CampusId'] == campus_id]
        
        self.buildings = buildings['BuildingId'].unique()
        
        self.Pri_entitled = self.FAS_1_2_entitlement_calc(Pri_FAS, Sec_FAS, self.enrolment_categories_tuple)[0] # !!! The FAS category data will need to have the PageOne class specified
        
        self.Sec_entitled = self.FAS_1_2_entitlement_calc(Pri_FAS, Sec_FAS, self.enrolment_categories_tuple)[1] # !!! The FAS category data will need to have the PageOne class specified
        
        self.FAS_2_entitlement = self.FAS_2_entitlement_calc(self.Pri_entitled, self.Sec_entitled, FAS_2_categories)[0]
        
        self.FAS_2_core_entitlement = self.FAS_2_entitlement_calc(self.Pri_entitled, self.Sec_entitled, FAS_2_categories)[1]
        
        self.FAS_1_core_entitlement = self.FAS_1_core_entitlement # <-- This is a pd.Series that shows the entitlement at the FAS 1 level
        
        self.FAS_1_entitlement = self.FAS_1_entitlement # <-- Same as above, except with non-core FS 2 categories filtered out
        
        self.entitled_core_space = self.FAS_1_core_entitlement.sum() # <-- This finds the sum of all the core space i.e  TOTAL CORE SPACE FOR THE CAMPUS
        
        self.buildings_dictionary = {}
        
        for ID in self.buildings:
            self.buildings_dictionary[ID] = Building(ID, building_properties, FAS_2_space, FAS_2_core_space, aims_extract)
            
            
        """
        Building attributes:
        1. self.FAS_2_space
        2. self.FAS_1_space
        3. self.FAS_1_core_space
        4. self.FAS_1_2_core_space
        
        The below section calculates the total & core space for the campus:
        """
        
        FAS_1_space_table_list = []
        FAS_1_core_space_table_list = []
        
        for i in self.buildings:
            FAS_1_space_table = self.buildings_dictionary[i].FAS_1_space
            FAS_1_space_table_list.append(FAS_1_space_table)
            
            FAS_1_core_space_table = self.buildings_dictionary[i].FAS_1_core_space
            FAS_1_core_space_table_list.append(FAS_1_core_space_table)
            
        self.FAS1_merged = pd.concat(FAS_1_space_table_list, axis = 0)
        self.FAS_1_space = self.FAS1_merged.sum()
        self.total_space = self.FAS_1_space.sum()

        self.FAS1_core_merged = pd.concat(FAS_1_core_space_table_list, axis = 1, join = 'outer')
        self.FAS1_core_merged.fillna(0, inplace=True)
        
        # This is a pd.Series that shows the core space for each FAS 1 category
        self.FAS_1_core_space = self.FAS1_core_merged.sum(axis = 1)
        
        # This is a float value that shows the sum of core space
        self.total_core_space = self.FAS_1_core_space.sum()
        
        """
        The below section calculates the excess core space for the campus:
        """
        
        actual = self.total_core_space
        entitled = self.entitled_core_space

        self.percentage_excess_core_space = ((actual - entitled) / actual ) * 100

        # This provides the excess floor area at each FAS 1 level
        self.FAS_1_excess_core_space = self.FAS_1_core_space.sub(self.FAS_1_core_entitlement, fill_value = 0)
        
        """
        The below section calculates the combinations of reducible buildings:
        """
        
        building_exclusions = inclusion_exclusion_df.loc[(inclusion_exclusion_df["table_name"] == "Building Properties") & (inclusion_exclusion_df["exclude_true_false"] == True)]
        
        numeric_exclusions = building_exclusions.loc[inclusion_exclusion_df["exclusion_type"] == 'num_range']
        
        string_exclusions = building_exclusions.loc[inclusion_exclusion_df["exclusion_type"] == 'string_value']
        
        exclusion_list = [] # <-- This is the list where excluded buildings are stored
        
        # iterates through the columns in the numeric exclusions included in the inclusion_exclusion dataframe
        for exclusion in numeric_exclusions["column_name"]:
            
            # checks if the exclusion is included in the building_properties dataframe
            if exclusion in list(buildings.columns):
                
                # defines the lower & upper thresholds for the exclusion criteria ( .iloc[0] is used to retrieve the first value from the pd.Series that is returned by .loc )
                low_value = numeric_exclusions.loc[numeric_exclusions["column_name"] == exclusion, "num_range_start"].iloc[0]
                high_value = numeric_exclusions.loc[numeric_exclusions["column_name"] == exclusion, "num_range_stop"].iloc[0]
                
                # Iterates through the buildings in the building_properties dataframe; actual_value is the number for the given building 
                for building in buildings["BuildingId"]:
                    actual_value = buildings.loc[buildings["BuildingId"] == building, exclusion].iloc[0]
                    
                    # checks if value is within exclusion range
                    try:
                        if actual_value >= low_value and actual_value <= high_value:
                            exclusion_list.append(building)
                    except:
                        print(f"Actual value: {type(actual_value)}, high-low: {high_value},{low_value}")
                        
        # iterates through the columns in the string_value exclusions included in the inclusion_exclusion dataframe
        for exclusion in string_exclusions["column_name"]:
            
            # checks if the exclusion is included in the building_properties dataframe
            if exclusion in list(buildings.columns):
                
                # This retrieve the comma-separated strings
                comma_separated_strings = string_exclusions.loc[string_exclusions["column_name"] == exclusion, "string_value_criteria"].iloc[0]
                
                # This breaks up the string into the individual and strips any spaces from the start/finish
                string_list = comma_separated_strings.split(",")
                string_list = [s.strip() for s in string_list]
                
                for building in buildings["BuildingId"]:
                    actual_value = buildings.loc[buildings["BuildingId"] == building, exclusion].iloc[0]
                    
                    # checks if the building attribute (actual_value) is in the list of exclusions
                    try:
                        if actual_value in string_list:
                            exclusion_list.append(building)
                    except:
                        print(f"Building: {building}, Actual value: {actual_value}, Exclusion list: {string_list}") # <-- This prints the attributes that caused an error to occur
        
        
        # This defines the list of buildings that are considered 'reducible'
        included_buildings = [b for b in buildings["BuildingId"] if b not in exclusion_list]
        
        print(included_buildings)
        
        """
        This section finds the combinations of reducible buildings
        """        

        """
        This converts negative values in the excess space FAS 1 table to zero:
        > This is so that we can detect whether the reduction of this combination would affect FAS 1 categories which are already under-entitled
        """ 
        
        # This finds all the combinations of reducible schools
        combinations_list = []
        for i in range(len(included_buildings)):
            combo = list(itertools.combinations(included_buildings, i+1))
            combinations_list.append(combo)
              
        # This converts the list into a continuous list of tuples, rather than a list of lists for each iteration
        combinations_list = [item for sublist in combinations_list for item in sublist]
        
        print(combinations_list)
 
        # this defines all the combinations within a dictionary using the format (combination tuple) : combination_object
        combinations_dict = {}
        for combo_tuple in combinations_list:
            combinations_dict[combo_tuple] = Combination(combo_tuple, self.FAS_1_excess_core_space, self.buildings_dictionary, self.FAS_1_core_entitlement, self.FAS_1_core_space, self.entitled_core_space)
        
        
        self.combinations = combinations_dict   
        
        combination_rank = self.find_highest_fas1_core_space(combinations_dict)
        
        self.reducible_space = combination_rank[1]
        
        self.reducible_combination = combination_rank[0]
        
        for k, v in combinations_dict.items():
            print(k, v.reducible_space)
        
        
    def find_highest_fas1_core_space(self, dictionary):
    
        highest_fas1_key = None
        highest_fas1_value = float('-inf') # <-- Sets the core space to negative infitinity to make sure that the first FAS 1 core space value is accepted

        for key, combination in dictionary.items():
            #print(key, combination)
            #print(combination.reducible_space)
            if combination.reducible_classification == True: # <-- tests whether the combination is classed as reducible or not
                if combination.reducible_space > highest_fas1_value:
                    #print(key, combination)
                    highest_fas1_key = key
                    highest_fas1_value = combination.reducible_space

        return highest_fas1_key, highest_fas1_value
    

    def enrolment_category(self,enrolment, max_val):
        
        # Generate a range of values at intervals of 25 using numpy
        bins = np.arange(0, max_val + 25, 25)
        if enrolment in bins:
            category = enrolment
            # Use numpy digitize to categorize the enrollment
        else:
            category_idx = np.digitize(enrolment, bins)
            # Return the highest value in the category
            category = bins[category_idx] if category_idx > 0 else None

        return category
    
    def enrolment_category1(self, enrolment, max_val):
        # Generate a range of values at intervals of 25 using numpy
        bins = np.arange(0, max_val + 25, 25)
        if enrolment in bins:
            category = enrolment
        else:
            category_idx = np.digitize(enrolment, bins)
            if category_idx < len(bins):
                category = bins[category_idx]
            else:
                category = None
        return category
    
    
    def enrolmentment_categories(self, campus_ID, campus_properties, FAS_1_categories, FAS_2_categories, Sec_FAS, Pri_FAS):
        
        # this gets the school based on the campus input, which is used to index the school_data dataframe
        school = campus_properties.loc[campus_properties['CampusId'] == campus_ID]
        
        # Converts the campus_id values in the campus properties to string values (should already be done but just a check)        
        
        primary_FAS_max_enrolments = (len(list(Pri_FAS.columns)) - 2) * 25 # this finds the number of the FAS categories (i.e. maximum number of primary students in the FAS)

        secondary_FAS_max_enrolments = (len(list(Sec_FAS.columns)) - 2) * 25 # this finds the number of the FAS categories (i.e. maximum number of secondary students in the FAS)

        primary_enrolments = campus_properties.loc[campus_properties['CampusId'] == campus_ID, "primary_enrolments"].item() # this retrieves the number of primary enrolments from the school_data for the given campus
        
        secondary_enrolments = campus_properties.loc[campus_properties['CampusId'] == campus_ID, "secondary_enrolments"].item() # this retrieves the number of secondary enrolments from the school_data for the given campus

        primary_enrolment_category = self.enrolment_category(primary_enrolments, primary_FAS_max_enrolments) # this finds the enrolment band for the primary FAS

        secondary_enrolment_category = self.enrolment_category(secondary_enrolments, secondary_FAS_max_enrolments) # this finds the enrolment band for the secondary FAS

        enrolment_categories_tuple = (str(primary_enrolment_category), str(secondary_enrolment_category))
        
        return enrolment_categories_tuple
    
    # This returns the entitled space for both primary and secondary FAS 2 categories as two dataframes in a tuple)
    def FAS_1_2_entitlement_calc(self, Pri_FAS, Sec_FAS, enrolment_categories):
        
        primary_enrolments_category = enrolment_categories[0]
        
        # Removes the enrolment bands from the Pri_FAS df and keeps the core/support classification
        Pri_entitled = Pri_FAS['Space classification']
        
        # Converts the pd.Series into a dataframe
        Pri_entitled = Pri_entitled.to_frame(name='Space classification')
        
        FAS_2_index = list(Pri_entitled.index)
        
        primary_entitlement_list = []
    
        # This loop finds the entitled space for each FAS 2 category, and has some (hopefully redundent) code for handling index duplicates and weird stuff
        for i in FAS_2_index:
            entitlement = Pri_FAS.loc[i, primary_enrolments_category]
            if type(entitlement) == pd.Series:
                entitlement = entitlement.mean()
            primary_entitlement_list.append(entitlement)
            
        Pri_entitled["Entitled Space"] = primary_entitlement_list
        
        secondary_enrolments_category = enrolment_categories[1]
        
        # Removes the enrolment bands from the Pri_FAS df and keeps the core/support classification
        Sec_entitled = Sec_FAS['Space classification']
        
        # Converts the pd.Series into a dataframe
        Sec_entitled = Sec_entitled.to_frame(name='Space classification')
        
        FAS_2_index = list(Sec_entitled.index)
        
        sec_entitlement_list = []
        
        for i in FAS_2_index:
            entitlement = Sec_FAS.loc[i, secondary_enrolments_category]
            if type(entitlement) == pd.Series:
                entitlement = entitlement.mean()
            sec_entitlement_list.append(entitlement)
            
        Sec_entitled["Entitled Space"] = sec_entitlement_list

        return Pri_entitled, Sec_entitled
    
    def FAS_2_entitlement_calc(self, pri_entitled, sec_entitled, FAS_2_categories):
        
        # This retrieves the FAS 2 categories from the FAS_2 categories dataframe
        FAS_2_entitlement = FAS_2_categories
        
        FAS_2_core_entitlement = FAS_2_categories.loc[FAS_2_categories["Space classification"] == "Core"]    
        
        idx = pd.IndexSlice # <--- This is used to deal with the multi-index in the pre/sec entitled dataframes
            
        entitlement_list = []
        
        for i in FAS_2_entitlement.index:
            entitled_space = 0
            if i in pri_entitled.index.get_level_values(1):
                entitlement = pri_entitled.loc[idx[:,i], "Entitled Space"].sum()
                entitled_space += entitlement
                
            if i in sec_entitled.index.get_level_values(1):
                entitlement = sec_entitled.loc[idx[:,i], "Entitled Space"].sum()
                entitled_space += entitlement
            #print(f"Entitlement: {i}; Area:", entitled_space) # <-- This can be used to check the result
            entitlement_list.append(entitled_space)
            
        FAS_2_entitlement["Entitled Space"] = entitlement_list
        
        core_entitlement_list = []
        
        for i in FAS_2_core_entitlement.index:
            entitled_space = 0
            if i in pri_entitled.index.get_level_values(1):
                entitlement = pri_entitled.loc[idx[:,i],"Entitled Space"].sum()
                entitled_space += entitlement
            if i in sec_entitled.index.get_level_values(1):
                entitlement = sec_entitled.loc[idx[:,i],"Entitled Space"].sum()
                entitled_space += entitlement
            #print(f"Entitlement: {i}; Area:", entitled_space)  # <-- This can be used to check the result
            core_entitlement_list.append(entitled_space)
        
        FAS_2_core_entitlement["Entitled Space"] = core_entitlement_list
        FAS_2_core_entitlement.loc[:, "Entitled Space"] = core_entitlement_list
        
        
        # The below section uses the FAS 2 entitlements to aggregate at the FAS 1 level
        
        self.primary_FAS_1_entitlement = pri_entitled.groupby(level=0)['Entitled Space'].sum()
        self.secondary_FAS_1_entitlement = sec_entitled.groupby(level=0)['Entitled Space'].sum()
        
        self.FAS_1_entitlement = self.primary_FAS_1_entitlement.add(self.secondary_FAS_1_entitlement, fill_value = 0)
        
        core_pri_entitled = pri_entitled.loc[pri_entitled["Space classification"] == "Core"]
        core_sec_entitled = sec_entitled.loc[sec_entitled["Space classification"] == "Core"]
        
        self.primary_FAS_1_core_entitlement = core_pri_entitled.groupby(level=0)['Entitled Space'].sum()
        self.secondary_FAS_1_core_entitlement = core_sec_entitled.groupby(level=0)['Entitled Space'].sum()
        
        self.FAS_1_core_entitlement = self.primary_FAS_1_core_entitlement.add(self.secondary_FAS_1_core_entitlement, fill_value = 0)
        
        return FAS_2_entitlement, FAS_2_core_entitlement
        
campus = Campus('1883001', campus_properties, actual_space_extract, FAS_2_space, FAS_1_space, building_properties, inclusion_exclusion_df)

building_1 = list(campus.buildings_dictionary.values())[0]

campus.FAS_1_excess_core_space

combinations = list(campus.combinations.items())

C:\Users\10349839\AppData\Local\Temp\ipykernel_10720\4275749806.py:345: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FAS_2_core_entitlement["Entitled Space"] = core_entitlement_list
C:\Users\10349839\AppData\Local\Temp\ipykernel_10720\4275749806.py:346: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FAS_2_core_entitlement.loc[:, "Entitled Space"] = core_entitlement_list
C:\Users\10349839\AppData\Local\Temp\ipykernel_10720\4275749806.py:345: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

['105585', '105304', '463586', '105303']
[('105585',), ('105304',), ('463586',), ('105303',), ('105585', '105304'), ('105585', '463586'), ('105585', '105303'), ('105304', '463586'), ('105304', '105303'), ('463586', '105303'), ('105585', '105304', '463586'), ('105585', '105304', '105303'), ('105585', '463586', '105303'), ('105304', '463586', '105303'), ('105585', '105304', '463586', '105303')]
ID: 105585, building space:                              Actual Space
FAS 1                                    
Administration                       38.8
School Community Hub                905.6
Student and Staff Amenities          98.0
Combination space:                              Actual Space  Administration  Learning Base  \
FAS 1                                                                      
Administration                        NaN             NaN            NaN   
School Community Hub                  NaN             NaN            NaN   
Student and Staff Amenities           NaN  

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [123]:
campus.FAS_1_excess_core_space

FAS 1
Administration                                    127.6
Learning Base                                     707.4
Learning Resource Centre                          363.0
Performing Arts, PE, Canteen, Food Technology      63.0
Resource Intensive Learning                       174.4
School Community Hub                             1056.6
Science                                          -174.0
Staff Work                                        112.0
Student and Staff Amenities                       269.0
Technology                                         61.6
The Arts - Fine Arts and Multimedia              -287.0
dtype: float64

In [141]:
campus.FAS_1_core_space

FAS 1
Administration                                    308.6
Learning Base                                    1285.4
Performing Arts, PE, Canteen, Food Technology      85.0
Resource Intensive Learning                       224.4
Staff Work                                        203.0
Student and Staff Amenities                       396.0
Technology                                        181.6
School Community Hub                             1056.6
Learning Resource Centre                          433.0
The Arts - Fine Arts and Multimedia                48.0
dtype: float64

In [143]:
combinations[5][1].combination_FAS_1_core_space

FAS 1
Administration                                   181.0
Learning Base                                    578.0
Learning Resource Centre                          70.0
Performing Arts, PE, Canteen, Food Technology     22.0
Resource Intensive Learning                       50.0
School Community Hub                               0.0
Science                                          174.0
Staff Work                                        91.0
Student and Staff Amenities                      127.0
Technology                                       120.0
The Arts - Fine Arts and Multimedia              335.0
Name: Entitled Space, dtype: float64

In [144]:
combinations[5][1].FAS_1_remaining_core_space

FAS 1
Administration                                    127.6
Learning Base                                     707.4
Learning Resource Centre                          363.0
Performing Arts, PE, Canteen, Food Technology      63.0
Resource Intensive Learning                       174.4
School Community Hub                             1056.6
Science                                             NaN
Staff Work                                        112.0
Student and Staff Amenities                       269.0
Technology                                         61.6
The Arts - Fine Arts and Multimedia              -287.0
dtype: float64

In [146]:
combinations[5][1].reducible_classification

False

In [5]:
    def enrolmentment_categories(self, campus_ID, campus_properties, FAS_1_categories, FAS_2_categories, Sec_FAS, Pri_FAS):
    
        # this gets the school based on the campus input, which is used to index the school_data dataframe
        school = campus_properties.index.get_level_values(0)[campus_properties.index.get_level_values(1) == campus_ID].item()

        primary_FAS_max_enrolments = (len(list(Pri_FAS.columns)) - 2) * 25 # this finds the number of the FAS categories (i.e. maximum number of primary students in the FAS)

        secondary_FAS_max_enrolments = (len(list(Sec_FAS.columns)) - 2) * 25 # this finds the number of the FAS categories (i.e. maximum number of secondary students in the FAS)

        primary_enrolments = campus_properties.loc[(school,campus), "Primary Enrolments"].item() # this retrieves the number of primary enrolments from the school_data for the given campus

        secondary_enrolments = campus_properties.loc[(school,campus), "Secondary Enrolments"].item() # this retrieves the number of secondary enrolments from the school_data for the given campus

        primary_enrolment_category = enrolment_category(primary_enrolments, primary_FAS_max_enrolments) # this finds the enrolment band for the primary FAS

        secondary_enrolment_category = enrolment_category(secondary_enrolments, secondary_FAS_max_enrolments) # this finds the enrolment band for the secondary FAS

        enrolment_categories = (primary_enrolment_category, secondary_enrolment_category)

        return enrolment_categories

In [6]:
"""Below is the method for processing the FAS1-2 Category dataframes"""
        
FAS_1_categories = pd.read_csv(r'FAS 1 Categories.csv', index_col = "Facility Area Schedule")
FAS_2_categories = pd.read_csv(r'FAS 2 Categories.csv', index_col = "Facility Area Schedule")
        
FAS_1_categories = FAS_1_categories
FAS_2_categories = FAS_2_categories
        
        
"""Below is the method for processing the Pri_FAS & Sec_FAS dataframes"""
        
Pri_FAS = Pri_FAS = pd.read_csv(r'Primary FAS.csv')
Sec_FAS = Sec_FAS = pd.read_csv(r'Secondary FAS.csv')

# 2: rename columns to match later requirements
Sec_FAS.rename(columns = {'Upper Enrolments': 'FAS Layer 2', 'FAS Level' : 'FAS Layer'}, inplace = True)
Pri_FAS.rename(columns = {'Upper Enrolments': 'FAS Layer 2'}, inplace = True)

# 3: Set the category names as the indices
Sec_FAS_index = Sec_FAS['FAS Layer 2']
Sec_FAS = Sec_FAS.set_index(Sec_FAS_index)
Pri_FAS_index = Pri_FAS['FAS Layer 2']
Pri_FAS = Pri_FAS.set_index(Pri_FAS_index)

# 4: remove FAS categories column
Sec_FAS = Sec_FAS.drop(['FAS Layer 2'], axis = 1)
Pri_FAS = Pri_FAS.drop(['FAS Layer 2'], axis = 1)

# 5: insert an enrolment band for '0' students
Pri_FAS.insert(1, '0', 0)
Sec_FAS.insert(1, '0', 0)
# 6: insert a column classifying which enrolment type the row belongs to

Pri_FAS.insert(1, 'Student Type', 'Primary')

Sec_FAS.insert(1, 'Student Type', 'Secondary')

# 7: Splits the indices into a multi-index depending on FAS Layer

#def split_index(FAS_df):
    #new_index = [(FAS_1 if (layer := FAS_df.iat[n, 0]) == 1 else FAS_1, index) for n, index in enumerate(FAS_df.index) if (FAS_1 := index) or layer == 2]
    #FAS_df.set_index(pd.MultiIndex.from_tuples(new_index, names=['FAS 1', 'FAS 2']), inplace=True)
    #FAS_df = FAS_df[FAS_df['FAS Layer'] > 1]
    #FAS_df = FAS_df.drop(columns=["FAS Layer"])

    #return FAS_df
    
def split_index(FAS_df):
    FAS_1 = None
    new_index = []
    
    for n, index in enumerate(FAS_df.index):
        layer = FAS_df.iat[n, 0]
        
        if layer == 1:
            FAS_1 = index
        elif layer == 2:
            new_index.append((FAS_1, index))
    
    FAS_df = FAS_df[FAS_df['FAS Layer'] > 1]
    FAS_df.set_index(pd.MultiIndex.from_tuples(new_index, names=['FAS 1', 'FAS 2']), inplace=True)
    FAS_df = FAS_df.drop(columns=["FAS Layer"])
    
    return FAS_df
    

Pri_FAS = split_index(Pri_FAS)

Sec_FAS = split_index(Sec_FAS)

# 8: Assigns the Core/Support label to each row (FAS Category)

def core_support(FAS_df, FAS_2_categories):
    core_support_column = []
    FAS_1_values = FAS_df.index.get_level_values(0).unique()
    FAS_2_values = FAS_df.index.get_level_values(1)
    for FAS_2 in FAS_2_values:
        if FAS_2_categories.at[FAS_2, "Space classification"] == 'Core':
            core_support_column.append('Core')
        elif FAS_2_categories.at[FAS_2, "Space classification"] == 'Support':
            core_support_column.append('Support')   
        else:
            core_support_column.append('N/A')
    FAS_df.insert(0, "Space classification", core_support_column)
    return FAS_df        
        
Pri_FAS = core_support(Pri_FAS, FAS_2_categories)
Sec_FAS = core_support(Sec_FAS, FAS_2_categories)

Pri_FAS.head(10)

Space classification  \
FAS 1                       FAS 2                                                                     
Learning Base               Shared Learning                                                    Core   
                            Practical Activities                                               Core   
                            Consult/Individual Learning Space                                  Core   
                            Support Spaces (Kitchen, Laundry, Equipment Store)                 Core   
                            Relocatable Buildings - Learning Base                              Core   
                            Circulation/Informal Learning                                   Support   
Resource Intensive Learning Practical Activities - Art/Science/Technology/F...                 Core   
                            Practical Activities - Science/Technology/Food ...                 Core   
                            Practical Activities - Art                                         Core   
                            Practical Activities - Science/Technology                          Core   

                                                                               Student Type  \
FAS 1                       FAS 2                                                             
Learning Base               Shared Learning                                         Primary   
                            Practical Activities                                    Primary   
                            Consult/Individual Learning Space                       Primary   
                            Support Spaces (Kitchen, Laundry, Equipment Store)      Primary   
                            Relocatable Buildings - Learning Base                   Primary   
                            Circulation/Informal Learning                           Primary   
Resource Intensive Learning Practical Activities - Art/Science/Technology/F...      Primary   
                            Practical Activities - Science/Technology/Food ...      Primary   
                            Practical Activities - Art                              Primary   
                            Practical Activities - Science/Technology               Primary   

                                                                                0  \
FAS 1                       FAS 2                                                   
Learning Base               Shared Learning                                     0   
                            Practical Activities                                0   
                            Consult/Individual Learning Space                   0   
                            Support Spaces (Kitchen, Laundry, Equipment Store)  0   
                            Relocatable Buildings - Learning Base               0   
                            Circulation/Informal Learning                       0   
Resource Intensive Learning Practical Activities - Art/Science/Technology/F...  0   
                            Practical Activities - Science/Technology/Food ...  0   
                            Practical Activities - Art                          0   
                            Practical Activities - Science/Technology           0   

                                                                                25  \
FAS 1                       FAS 2                                                    
Learning Base               Shared Learning                                     65   
                            Practical Activities                                 7   
                            Consult/Individual Learning Space                    8   
                            Support Spaces (Kitchen, Laundry, Equipment Store)   0   
                            Relocatable Buildings - Learning Base                0   
                            Circulation/Informal Learning                     

In [64]:
FAS_2_categories.loc[FAS_2_categories["Space classification"] == "Core"]

,Student Type,Space classification,Entitled Space
Facility Area Schedule,,,
Shared Learning,Pri-Sec,Core,370
Practical Activities,Pri-Sec,Core,34
Consult/Individual Learning Space,Pri-Sec,Core,24
"Support Spaces (Kitchen, Laundry, Equipment Store)",Pri-Sec,Core,0
Relocatable Buildings - Learning Base,Pri-Sec,Core,150
Practical Activities - Art/Science/Technology/Food Preparation,Primary,Core,50
Practical Activities - Science/Technology/Food Preparation,Primary,Core,0
Practical Activities - Art,Primary,Core,0
Practical Activities - Science/Technology,Primary,Core,0


In [54]:
buildings_test = ['463586', '105303', '120356']

combinations = []

for i in range(len(buildings_test)):
        combinations += list(itertools.combinations(buildings_test, i+1))
        
combinations

[('463586',),
 ('105303',),
 ('120356',),
 ('463586', '105303'),
 ('463586', '120356'),
 ('105303', '120356'),
 ('463586', '105303', '120356')]